In [131]:
# Imports
import pandas as pd
import time 
import math

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

In [132]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'uci-data/'
MODELS_PATH = DATA_PATH + 'models/raw-models/'
FEATURES = ['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]

BATCH_SIZE = 32
SAMPLE_SIZE = BATCH_SIZE * 1500
EPOCHS = 25

In [133]:
'''
# Load the data
train_set = pd.read_csv(DATA_PATH + "self-calculated/raw-train-data.txt", sep='\s+', header=None)
test_set = pd.read_csv(DATA_PATH + "self-calculated/raw-test-data.txt", sep='\s+', header=None)

train_set.columns=FEATURES
test_set.columns=FEATURES
'''
complete_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data.txt", sep='\s+', header=None)
complete_set.columns = FEATURES
train_set, test_set = train_test_split(complete_set, test_size=0.3)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
/var/folders/lt/w0169b7x5ml3psz3nly9vj3m0000gn/T/ipykernel_43370/1212202754.py:1: SyntaxWarning: invalid escape sequence '\s'
  '''
/var/folders/lt/w0169b7x5ml3psz3nly9vj3m0000gn/T/ipykernel_43370/1212202754.py:9: SyntaxWarning: invalid escape sequence '\s'
  complete_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data.txt", sep='\s+', header=None)


In [134]:
# Creating model
device = None

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

class RawDataModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.sequential_module = nn.Sequential(
            nn.Conv1d(in_channels=7, out_channels=256, kernel_size=1),
            nn.ReLU(), 
            nn.MaxPool1d(1),
            nn.BatchNorm1d(256),

            nn.Conv1d(in_channels=256, out_channels=128, kernel_size=1),
            nn.ReLU(), 
            nn.MaxPool1d(1),
            nn.BatchNorm1d(128),

            nn.Conv1d(in_channels=128, out_channels=64, kernel_size=1),
            nn.ReLU(), 
            nn.MaxPool1d(1),
            nn.BatchNorm1d(64),
            
            nn.Flatten(),

            nn.Linear(64, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(),

            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(),

            nn.Linear(128, 12),
            nn.Softmax()
        )

    def forward(self, x):
        return self.sequential_module(x)
    
model = RawDataModel().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [135]:
# Training and testing function
def train(dataloader, model, loss_fn, optimizer):
    # Get batch num
    num_batches = len(dataloader.dataset) / BATCH_SIZE
    i = 0

    # Set the model to train mode
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X = X.unsqueeze(-1)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        i += 1
        if batch % 50 == 0:
            print(f"loss: {loss.item()}, batch: {i} out of {math.ceil(num_batches)}")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.unsqueeze(-1)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    return(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [136]:
# Dataset preparation
class HAPTDataset(Dataset):
    def __init__(self, dataset, features, label):
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)[:SAMPLE_SIZE]
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)[:SAMPLE_SIZE]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
train_dataset = HAPTDataset(train_set, train_set.columns[:-1], 'activity')
test_dataset = HAPTDataset(test_set, test_set.columns[:-1], 'activity')

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [137]:
perf_timer = time.perf_counter()
perf_acc = ""

for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    perf_acc = test(test_dataloader, model, loss_function)
    
print(perf_acc)
perf_timer = time.perf_counter() - perf_timer
print("Done!")

Epoch 1
-------------------------------
loss: 2.515962600708008, batch: 1 out of 1500


/opt/homebrew/lib/python3.12/site-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 2.419985771179199, batch: 51 out of 1500
loss: 2.354149341583252, batch: 101 out of 1500
loss: 2.2254412174224854, batch: 151 out of 1500
loss: 2.237992286682129, batch: 201 out of 1500
loss: 2.153644561767578, batch: 251 out of 1500
loss: 2.0862958431243896, batch: 301 out of 1500
loss: 2.0832762718200684, batch: 351 out of 1500
loss: 1.9158828258514404, batch: 401 out of 1500
loss: 1.984983205795288, batch: 451 out of 1500
loss: 2.1097092628479004, batch: 501 out of 1500
loss: 1.9958930015563965, batch: 551 out of 1500
loss: 2.064727783203125, batch: 601 out of 1500
loss: 2.021512985229492, batch: 651 out of 1500
loss: 2.0890917778015137, batch: 701 out of 1500
loss: 2.0066006183624268, batch: 751 out of 1500
loss: 2.108241558074951, batch: 801 out of 1500
loss: 2.1001148223876953, batch: 851 out of 1500
loss: 2.0084948539733887, batch: 901 out of 1500
loss: 2.1027328968048096, batch: 951 out of 1500
loss: 2.045600175857544, batch: 1001 out of 1500
loss: 2.001514434814453, batc

In [138]:
model_name = "cnn_11"
if True:
    torch.save(model.state_dict(), MODELS_PATH + model_name + ".pth")

    with open(MODELS_PATH + model_name + ".txt", "w") as f:
        f.write("Epochs: {}\n".format(EPOCHS))
        f.write("Batch Size: {}\n".format(BATCH_SIZE))
        f.write("Sample Size: {}\n".format(SAMPLE_SIZE))
        f.write("Feature Set: {}\n".format(FEATURES))
        f.write("Model: {}\n".format(str(model)))
        f.write("Loss Function: {}\n".format("Cross Entropy Loss"))
        f.write("Optimizer: {}\n\n\n\n".format(str(optimizer)))
        f.write("Results: {}\n".format(perf_acc))
        f.write("Timer: {}s\n".format(round(perf_timer, 2)))